In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np  # pd.get_dummies() -> DataFrame 반환 -> array로 type 변경하기 위함

In [2]:
# 8x4 (4개의 특징을 가진 데이터 row 8개)
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

In [3]:
# 0, 1, 2  : 총 3개의 클래스 존재
y_train = [2, 2, 2, 1, 1, 1, 0, 0]   # GT=Label Data,  각 샘플([1, 2, 1, 1] ==> 2)에 대한 GT 값

# ▶ one-hot encoding으로 y_train을 변경
1) pandas의 get_dummies() 메서드 사용
2) torch : one_hot() 변경하는 메서드 없음  => 파이토치 시작할 때 사용한 scatter, unsqueeze  메서드 이용해서 구현
3) nn.Linear(), F.CrossEntropy  사용시 Y_train을 one_hot encoding하지 않아도 자동으로 변환

## 1. DataFrame 형태의 GT

In [4]:
df = pd.DataFrame({"y_train": y_train})
df

,y_train
0,2
1,2
2,2
3,1
4,1
5,1
6,0
7,0


## 2. pd.get_dummies 로 원핫인코딩

In [5]:
df = pd.get_dummies(df["y_train"])
df

,0,1,2
0,0,0,1
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0
5,0,1,0
6,1,0,0
7,1,0,0


## 3. np.array로 변환
DataFrame => array (8x3) 변환해줘야 학습 가능

In [6]:
new_array = np.array(df)
new_array

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

## 4. pytorch에서 사용하기 위해서 Tensor 로 변경 

In [7]:
y_train_new = torch.FloatTensor(new_array)
y_train_new.shape, y_train_new

(torch.Size([8, 3]),
 tensor([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.]]))

# ▶ scatter, unsqueeze를 이용한 one-hot encoding 구현

원핫인코딩을 하기 위해서는 사이즈를 만들어줘야 함
- y_one_hot의 사이즈를 정의
- row가 8개 있고, class가 3개 있음 => 8x3 one-hot encoding된 데이터 필요

In [8]:
# torch 메서드들을 사용하기 위해 tensor로 변경하기
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)  # scatter_() 메서드 사용을 위해 LongTensor로 변경

In [9]:
# y_train 원핫인코딩하기 위한 준비
y_one_hot = torch.zeros(8, 3)
y_one_hot

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

## ▷ unsqueeze()
- unsqueeze() 전 y_train.shpae 은 (8, ) => unsqueeze() 후 : (8x1)  

- 0은 row, 1은 col으로 확장 : shpae이 (8)일 때,
    - unsqueeze(0) => 1x8
    - unsqueeze(1) => 8x1

cf) squeeze (제거하다) : (8x1) -> (8)

In [10]:
# unsqueeze() 전
y_train.shape, y_train

(torch.Size([8]), tensor([2, 2, 2, 1, 1, 1, 0, 0]))

In [11]:
# unsqueeze() 후
y_train.unsqueeze(1).shape, y_train.unsqueeze(1)  # 컬럼 방향으로 확장

(torch.Size([8, 1]),
 tensor([[2],
         [2],
         [2],
         [1],
         [1],
         [1],
         [0],
         [0]]))

## ▷ scatter (흩어지다) : 열방향 또는 행 방향으로 흩어짐
```python
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
```
- 1번째 파라미터 : 어느 dim 으로 확장할 것인지 결정
- 3번째 파라미터 : y_train값에 해당하는 인덱스에 1을 삽입 (y_train이 1이면 0,1,0 됨)
- _ 언더바 : inplace=True와 같은 개념

In [12]:
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
y_one_hot.shape, y_one_hot

(torch.Size([8, 3]),
 tensor([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.]]))

In [13]:
y_one_hot.scatter(1, y_train.unsqueeze(1), 3)  # 3번째 파라미터 : y_train값에 해당하는 인덱스에 3을 삽입

tensor([[0., 0., 3.],
        [0., 0., 3.],
        [0., 0., 3.],
        [0., 3., 0.],
        [0., 3., 0.],
        [0., 3., 0.],
        [3., 0., 0.],
        [3., 0., 0.]])

# ▶ 학습 하기 위한 준비
1. weight, bias shape 정의
2. optimizer를 뭘 사용할 것인지 등을 정의
3. for문에서 정의된 epoch 수만큼 돌면서 w, b 값 학습  
    3-1. 가설함수(hypothesis: softmax)  
    3-2. cost function: cross-entropy  

# weight 설정
```python
w = torch.zeros((), requires_grad=True)  # ()에 shape 들어가야 함
```
- x_train.shape : 8x4  
- y_train.shape : 8x3  
- ```x_train * w => y_train``` 나와야 함


### 8x4 matrix multiplication 4x3 => 8x3

In [14]:
w = torch.zeros((4, 3), requires_grad=True) # requires_grad : w 기준으로 편미분
b = torch.zeros(3, requires_grad=True)  # class가 3개니까??열이 3개니까?? 3개 나와야 함
# bias는 wx+b로 더하는 값임. 그래서 크게 영향을 미치지는 않음

In [15]:
# z = torch.FloatTensor([1,2,3])
# torch.exp(z[0]/(torch.exp(z[0] + torch.exp(z[1] + torch.exp(z[2])))
# F.softmax(z)

## 방법 1

In [17]:
optimizer = optim.SGD([w, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 1. Hypothesis 함수 : forward
    # softmax = exp(wx+b) / sum(exp(wx+b))
    # => 구현되어 있는게 F.softmax
    hx = F.softmax(x_train.matmul(w) + b, dim=1)
    
    # 2. Cost Function 사용
    # cross-entropy = 1 / n(y*-log(h(x)))  : *는 element-wise 곱
    cost = (y_one_hot * -torch.log(hx)).sum(dim=1).mean()  # y_one_hot는 GT값
    
    # 3. 최적화 : cost function을 미분하고, 경사타고 내려오면서 w,b 업데이터
    optimizer.zero_grad()
    cost.backward()  # cost function 미분
    optimizer.step()  # SGD, lr 만큼 내려가면서 w, b 업데이트
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs}, cost: {cost.item():.6f}")

Epoch    0/1000, cost: 1.098612
Epoch  100/1000, cost: 0.704199
Epoch  200/1000, cost: 0.623000
Epoch  300/1000, cost: 0.565717
Epoch  400/1000, cost: 0.515291
Epoch  500/1000, cost: 0.467662
Epoch  600/1000, cost: 0.421278
Epoch  700/1000, cost: 0.375401
Epoch  800/1000, cost: 0.329766
Epoch  900/1000, cost: 0.285072
Epoch 1000/1000, cost: 0.248155


## 방법 2

In [19]:
model = nn.Linear(4, 3)  # 4x3 입력될 특징 4가지, 출력될 값 3가지  # 추가된 부분
optimizer = optim.SGD(model.parameters(), lr=0.1)
# optimizer = optim.SGD([w, b], lr=0.1) 사용하면 우리가 만든 zeros로 계속 학습하기 때문에 cost가 업데이트 안됨!

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 1. Hypothesis 함수 : forward
    # hx = F.softmax(x_train.matmul(w) + b, dim=1)
    
    # 2. Cost Function 사용
    # cost = (y_one_hot * -torch.log(hx)).sum(dim=1).mean()  # y_one_hot는 GT값
    
    # 방법1과 다른 부분
    # 1, 2 대신 아래 두 줄로 변경해도 됨. 1, 2번의 코드(hx, cost)는 dim 등 직접 구현해서 이해하기 위함
    pred = model(x_train)
    cost = F.cross_entropy(pred, y_train)  
    # y_train는 원핫인코딩 안된 값. cross_entropy가 자동으로 원핫인코딩 해줌
    # cross_entropy 안에 softmax 들어있음
    # (우리가 구현한 softmax와 다름. 로그 먹인 softmax)
    
    # 3. 최적화 : cost function을 미분하고, 경사타고 내려오면서 w,b 업데이터
    optimizer.zero_grad()
    cost.backward()  # cost function 미분
    optimizer.step()  # SGD, lr 만큼 내려가면서 w, b 업데이트
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs}, cost: {cost.item():.6f}")

Epoch    0/1000, cost: 1.837496
Epoch  100/1000, cost: 0.648946
Epoch  200/1000, cost: 0.561758
Epoch  300/1000, cost: 0.506929
Epoch  400/1000, cost: 0.462533
Epoch  500/1000, cost: 0.423128
Epoch  600/1000, cost: 0.386321
Epoch  700/1000, cost: 0.350591
Epoch  800/1000, cost: 0.314768
Epoch  900/1000, cost: 0.278281
Epoch 1000/1000, cost: 0.245994
